In [1]:
pip install requests beautifulsoup4 pandas openpyxl

   ---------------------------------------- 0.0/250.9 kB ? eta -:--:--
   ------------------- -------------------- 122.9/250.9 kB 3.6 MB/s eta 0:00:01
   ---------------------------------------  245.8/250.9 kB 5.0 MB/s eta 0:00:01
   ---------------------------------------- 250.9/250.9 kB 2.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# This is to create an excel file.
"""
import pandas as pd

data = {
    "Website_URLs": [
        "https://www.example.com",
        "https://www.wikipedia.org",
        "https://www.python.org"
    ]
}

excel_file_path = "website_urls.xlsx"
df = pd.DataFrame(data)
df.to_excel(excel_file_path, index=False)
print(f"Excel file with URLs created at: {excel_file_path}")
"""

'\nimport pandas as pd\n\ndata = {\n    "Website_URLs": [\n        "https://www.example.com",\n        "https://www.wikipedia.org",\n        "https://www.python.org"\n    ]\n}\n\nexcel_file_path = "website_urls.xlsx"\ndf = pd.DataFrame(data)\ndf.to_excel(excel_file_path, index=False)\nprint(f"Excel file with URLs created at: {excel_file_path}")\n'

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Function to fetch and clean data
def fetch_and_clean_data(url):
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()  # Raise an error for bad status codes
        soup = BeautifulSoup(response.content, "html.parser")
        cleaned_text = soup.get_text(separator=" ", strip=True)
        return cleaned_text
    except Exception as e:
        return f"Error fetching URL {url}: {e}"

# Read URLs from the Excel file
input_file = "website_urls.xlsx"
df = pd.read_excel(input_file)

# Process each URL
cleaned_data = []
for url in df["Website_URLs"]:
    cleaned_data.append(fetch_and_clean_data(url))

# Save the cleaned data to a new Excel file
df["Cleaned_Text"] = cleaned_data
output_file = "cleaned_website_data.xlsx"
df.to_excel(output_file, index=False)
print(f"Cleaned data saved at: {output_file}")


Cleaned data saved at: cleaned_website_data.xlsx


In [4]:
import pandas as pd
from bs4 import BeautifulSoup
import re

# Load the Excel file
file_path = "cleaned_website_data.xlsx"  # Replace with your actual file path
df = pd.read_excel(file_path)

In [5]:
# Define the preprocessing function
def preprocess_text(raw_text):
    try:
        # Step 1: Remove HTML tags and scripts
        soup = BeautifulSoup(raw_text, "html.parser")
        cleaned_text = soup.get_text(separator=" ", strip=True)

        # Step 2: Normalize whitespace
        cleaned_text = re.sub(r'\s+', ' ', cleaned_text)

        # Step 3: Remove special characters
        cleaned_text = re.sub(r'[^a-zA-Z0-9\s]', '', cleaned_text)

        # Step 4: Convert to lowercase
        cleaned_text = cleaned_text.lower()

        # Step 5 (Optional): Tokenization (if needed, uncomment below)
        # tokens = cleaned_text.split()

        # Step 6 (Optional): Remove stopwords (if needed, uncomment below)
        # from nltk.corpus import stopwords
        # stop_words = set(stopwords.words('english'))
        # tokens = [word for word in tokens if word not in stop_words]
        # cleaned_text = " ".join(tokens)

        return cleaned_text

    except Exception as e:
        return f"Error processing text: {e}"

In [10]:
# Apply preprocessing to the "Cleaned_Text" column
processed_df = df[['Cleaned_Text']].copy()
processed_df['Processed_Text'] = processed_df['Cleaned_Text'].apply(preprocess_text)

# Save the processed data to a separate Excel file
output_file = "processed_website_data.xlsx"
processed_df.to_excel(output_file, index=False)
print(f"Processed data saved to: {output_file}")


Processed data saved to: processed_website_data.xlsx


C:\Users\C-Dac\AppData\Local\Temp\ipykernel_10632\663602529.py:5: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(raw_text, "html.parser")


In [6]:
### 1. Generate Embeddings

In [7]:
from sentence_transformers import SentenceTransformer
import pandas as pd

# Load processed data
file_path = "processed_website_data.xlsx"
df = pd.read_excel(file_path)

In [8]:
# Load embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings
df['Embeddings'] = df['Processed_Text'].apply(lambda text: model.encode(text).tolist())

# Save embeddings
df.to_pickle("processed_data_with_embeddings.pkl")

c:\Users\C-Dac\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [11]:
import faiss
import numpy as np

In [12]:
# Load embeddings
df = pd.read_pickle("processed_data_with_embeddings.pkl")
embeddings = np.array(df['Embeddings'].tolist(), dtype='float32')

In [13]:
# Create and save FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)  # L2 distance metric
index.add(embeddings)
faiss.write_index(index, "semantic_search_index.faiss")

In [14]:
# Generate query embedding
query = "Latest cricket news"
query_embedding = model.encode(query).reshape(1, -1)

In [16]:
# Search the index
D, I = index.search(query_embedding, k=5)  # Retrieve top 5 results
results = df.iloc[I[0]]
print(results[['Processed_Text']])

                                       Processed_Text
21  todays cricket match  cricket update  cricket ...
0   live cricket score schedule latest news stats ...
15  sporting news india  nba  cricket  football  t...
10  bbc sport  scores fixtures news  live sport bb...
8   the athletic uk  sports news commentary result...
